In [2]:
this_file_name = 'testing_mapping_between_frames.ipynb'
import sys
import os
sys.path.insert(0, os.path.abspath(
    os.path.join(os.path.dirname(this_file_name), '../../src/') ))

from configuration_space_mapping import *
from utils import *

## How to map the vertices to global reference frame?


### First, rotation matrix
Well, we have the euler angles, right? [We can create a rotation matrix using them](http://eecs.qmul.ac.uk/~gslabaugh/publications/euler.pdf). Consider the angles $\phi$ (rotation around Z) , $\theta$ (rotation around Y) and $\psi$ (rotation around X). The rotation matrix is given by:

$$
R = R_z(\phi) R_y(\theta) R_x(\psi) = 
\begin{bmatrix}
cos \theta cos \phi & sin \psi sin \theta cos \phi - cos \psi sin \phi & cos \psi sin \theta cos \phi + sin \psi sin \phi \\
cos \theta sin \phi & sin \psi sin \theta sin \phi + cos \psi cos \phi & cos \psi sin \theta sin \phi - sin \psi cos \phi \\
-sin \theta & sin \psi cos \theta & cos \psi cos \theta \\
\end{bmatrix}.
$$

This means that, if we get a 3D point $\vec{x}$ and multiply by $R$, we get the same effect of rotating $\vec{x}$ $\phi$ around Z, $\theta$ around Y and $\psi$ around X. In order to get the point back to the original frame, we take the inverse of R.

In [3]:
import numpy as np
# testing the forward transform: vector = [1, 0, 0] and a rotation of 90º around Z axis
mat = np.round( rotation_matrix_from_euler_angles(3.14/2,0,0, False), 2 )
print(mat)
vector = np.array([1,0,0])
np.matmul(mat,vector) # vector goes to [0, 1, 0]

[[ 0. -1.  0.]
 [ 1.  0.  0.]
 [-0.  0.  1.]]


array([0., 1., 0.])

In [4]:
# testing the inverse: the vector is at [0, 1, 0], and we know there happened a Z axis 90º rotation. Where was the vector?
mat = np.round( rotation_matrix_from_euler_angles(3.14/2,0,0, True), 2 )
print(mat)
vector = np.array([0,1,0])
np.matmul(mat,vector) # obviously the vector was at [1, 0, 0]

[[ 0.  1.  0.]
 [-1.  0. -0.]
 [ 0.  0.  1.]]


array([1., 0., 0.])

In [5]:
# Ok, now a more case-specific test. Consider we have two reference frames, a global one and the robot's. 
# The only difference is that the robot's reference frame is rotated -90º around the Z axis. If a point on 
# robot's reference frame is equal to [1, 0, 0], where does it lie on the global reference frame?
# Naturally, we would say it lies on [0, -1, 0]. Which transformation should we use? Actually, the forward one.
 
mat = np.round( rotation_matrix_from_euler_angles(-3.14/2,0,0, False), 2 )
print(mat)
vector = np.array([[1],[0],[0]])
np.matmul(mat,vector) # see?

[[ 0.  1.  0.]
 [-1.  0. -0.]
 [-0.  0.  1.]]


array([[ 0.],
       [-1.],
       [ 0.]])

### Second: we just need to add the robot's frame origin position 

In [6]:
# Let's test how we can get a given point (originally in robot's frame) to global frame. Suppose we have
# a robot's frame whose origin's coordinates are [0,10,0] (in global reference frame). Suppose a 180º 
# rotation (around Z axis) happened. There is a bounding box vertice at [1,1,0] (robot's frame). Where
# is such vertice in global frame?
# After the proper calculations we deduce it lies on [-1, 9, 0].

In [7]:
vector = np.array([[1],[1],[0]])
frame_origin = np.array([[0],[10],[0]])
euler_a = [3.14,0,0]
np.round(get_global_position(vector,euler_a,frame_origin),2) # see?

array([[-1.],
       [ 9.],
       [ 0.]])